In [1]:
using Distributed, Plots, GaussQuadrature, SparseArrays, DataFrames, LinearAlgebra, Random, BenchmarkTools, CUDA, StaticArrays

CUDA.allowscalar()
# CUDA.allowscalar(false)

@everywhere using CUDA

┌ Warning: It's not recommended to use allowscalar([true]) to allow scalar indexing.
│ Instead, use `allowscalar() do end` or `@allowscalar` to denote exactly which operations can use scalar operations.
└ @ GPUArraysCore /home/pedro/.julia/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:188


In [2]:
function exemplo1()
	α = 1.0
	β = 1.0
	f(x₁,x₂) = (2*α*π^2+β) * sin(π*x₁) * sin(π*x₂)
	u(x₁,x₂) = sin(π*x₁) * sin(π*x₂)

	return α, β, f, u
end

exemplo1 (generic function with 1 method)

# Função $\varphi$

In [3]:
function ϕ(ξ₁,ξ₂,a)
	if a == 1
		return (1-ξ₁)*(1-ξ₂)/4
	elseif a == 2
		return (1+ξ₁)*(1-ξ₂)/4
	elseif a == 3
		return (1+ξ₁)*(1+ξ₂)/4
	elseif a == 4
		return (1-ξ₁)*(1+ξ₂)/4
	else
		error("a deve ser 1, 2, 3 ou 4.")
	end
end


function ϕ(ξ₁::Float64, ξ₂::Float64) :: Vector{Float64}
	[(1-ξ₁)*(1-ξ₂)/4, (1+ξ₁)*(1-ξ₂)/4, (1+ξ₁)*(1+ξ₂)/4, (1-ξ₁)*(1+ξ₂)/4]
end

ϕ (generic function with 2 methods)

# Derivadas de $\varphi$

In [4]:
@inline function ∂ϕ_∂ξ₁(ξ₁::Float64,ξ₂::Float64,a::Int)
	val = ifelse(a == 1, -(1-ξ₂)/4,
           ifelse(a == 2, (1-ξ₂)/4,
           ifelse(a == 3, (1+ξ₂)/4,
           ifelse(a == 4, -(1+ξ₂)/4, NaN))))
    return val
end

@inline function ∂ϕ_∂ξ₁(ξ₂::Float64) :: Vector{Float64}
	[-(1-ξ₂)/4, (1-ξ₂)/4, (1+ξ₂)/4, -(1+ξ₂)/4]
end

∂ϕ_∂ξ₁ (generic function with 2 methods)

In [5]:
@inline function ∂ϕ_∂ξ₂(ξ₁::Float64, ξ₂::Float64, a::Int)
    # Replace `if-elseif` with a `switch`-like computation for CUDA
    val = ifelse(a == 1, -(1-ξ₁)/4,
           ifelse(a == 2, -(1+ξ₁)/4,
           ifelse(a == 3, (1+ξ₁)/4,
           ifelse(a == 4, (1-ξ₁)/4, NaN))))
    return val
end

@inline function ∂ϕ_∂ξ₂(ξ₁::Float64) :: Vector{Float64}
	[-(1-ξ₁)/4, -(1+ξ₁)/4, (1+ξ₁)/4, (1-ξ₁)/4]
end

∂ϕ_∂ξ₂ (generic function with 2 methods)

# Mudança de variável

In [6]:
@inline function x₁_de_ξ(ξ₁::Float64, h₁::Float64, p₁::Float64)::Float64
    return p₁ + (h₁ / 2) * (ξ₁ + 1)
end

@inline function x₂_de_ξ(ξ₂::Float64, h₂::Float64, p₂::Float64)::Float64
    return p₂ + (h₂ / 2) * (ξ₂ + 1)
end

x₂_de_ξ (generic function with 1 method)

# Montagem da F local

In [12]:

function monta_Fᵉ!(Fᵉ::Vector{Float64}, f::Function, h₁::Float64, h₂::Float64, p₁::Float64, p₂::Float64, P::Vector{Float64}, W::Vector{Float64})
    # Zera as entradas do vetor local Fᵉ
    fill!(Fᵉ, 0.0)

	# Pré-calcula o fator constante da integral
	cst = h₁ * h₂ / 4

    phi_values = precompute_ϕ(P)

    # Loop sobre as entradas do vetor local Fᵉ
    for a in 1:4
        # Inicializa a contribuição da quadratura para a entrada a
        soma = 0.0
        
        # Integração via quadratura gaussiana dupla
        for i in 1:length(P)          # Loop sobre os pontos de quadratura no eixo ξ₁
			ξ₁ = P[i]
            x₁ = x₁_de_ξ(ξ₁, h₁, p₁)
            for j in 1:length(P)      # Loop sobre os pontos de quadratura no eixo ξ₂
				ξ₂ = P[j]
                x₂ = x₂_de_ξ(ξ₂, h₂, p₂)

                # Acumula a contribuição da quadratura
                #soma += W[i] * W[j] * f(x₁, x₂) * ϕ(ξ₁, ξ₂, a) * cst
                soma += W[i] * W[j] * f(x₁, x₂) * phi_values[i,j,a] * cst
            end
        end
        
        # Armazena a contribuição no vetor Fᵉ
        Fᵉ[a] = soma
    end
end

function precompute_ϕ_flat(P::Vector{Float64})::Vector{Float64}
    n = length(P)
    phi_flat = Vector{Float64}(undef, 4 * n^2)

    for a in 1:4
        for i in 1:n
            ξ₁ = P[i]
            for j in 1:n
                ξ₂ = P[j]
                index = (a - 1) * n^2 + (i - 1) * n + (j - 1) + 1
                if a == 1
                    phi_flat[index] = (1 - ξ₁) * (1 - ξ₂) / 4
                elseif a == 2
                    phi_flat[index] = (1 + ξ₁) * (1 - ξ₂) / 4
                elseif a == 3
                    phi_flat[index] = (1 + ξ₁) * (1 + ξ₂) / 4
                elseif a == 4
                    phi_flat[index] = (1 - ξ₁) * (1 + ξ₂) / 4
                end
            end
        end
    end

    return phi_flat
end

function monta_Fᵉ_cuda_correct!(Fᵉ, f::Function, h₁::Float64, h₂::Float64, p₁::Float64, p₂::Float64, P, W, phi_values)
    # Zera as entradas do vetor local Fᵉ
    CUDA.fill!(Fᵉ, 0.0)

	cst = h₁ * h₂ / 4

    # GPU kernel definition
    kernel_monta_Fᵉ = nothing
    function kernel_monta_Fᵉ!(
        Fᵉ, f::Function, h₁::Float64, h₂::Float64, p₁::Float64, p₂::Float64, P::CuDeviceVector{Float64, 1}, W::CuDeviceVector{Float64, 1}, cst::Float64
    )
        a = threadIdx().x
        soma = 0.0
        length_P = length(P)
        for i in 1:length_P     # Loop over quadrature points in ξ₁
            ξ₁ = P[i]
            x₁ = x₁_de_ξ(ξ₁, h₁, p₁)
            for j in 1:length_P # Loop over quadrature points in ξ₂
                ξ₂ = P[j]
                x₂ = x₂_de_ξ(ξ₂, h₂, p₂)
                # Accumulate the quadrature contribution
                index = (a - 1) * length_P^2 + (i - 1) * length_P + (j - 1) + 1
                soma += W[i] * W[j] * f(x₁, x₂) * phi_values[index] * cst
            end
        end
        # Store the result in Fᵉ
        Fᵉ[a] = soma
        return nothing
    end

    # Launch the kernel with 4 threads (one for each `a`)
    @cuda threads=4 kernel_monta_Fᵉ!(Fᵉ, f, h₁, h₂, p₁, p₂, P, W, cst)
end

monta_Fᵉ_cuda_correct! (generic function with 1 method)

In [13]:
function teste_monta_Fᵉ_cuda_correct()
	Fᵉ = CUDA.zeros(4)
	
	P1, W1 = legendre(5)
	phi_values = CuArray(precompute_ϕ_flat(P1))
    P, W = CuArray(P1), CuArray(W1)
	
	h₁ = 1/4
	h₂ = 1/4
	
	p₁ = 0.0
	p₂ = 0.0
	
	@inline function f(x₁,x₂) 
		return 4/(h₁*h₂)
	end

	#monta_Fᵉ_cuda_correct!(Fᵉ, f, h₁, h₂, p₁, p₂, P, W, phi_values)
    #display("Fᵉ - Teste 1")
	#display(Fᵉ)
	
	monta_Fᵉ_cuda_correct!(Fᵉ, (x₁,x₂) -> (16*9*x₁*x₂)/((h₁*h₂)^2), h₁, h₂, p₁, p₂, P, W, phi_values)
    display("Fᵉ - Teste 2")
	display(Fᵉ)
end

teste_monta_Fᵉ_cuda_correct (generic function with 1 method)

In [14]:
teste_monta_Fᵉ_cuda_correct()

"Fᵉ - Teste 2"

4-element CuArray{Float32, 1, CUDA.DeviceMemory}:
  4.0
  8.0
 16.0
  8.0

# Montagem da K local

In [260]:
function monta_Kᵉ(α::Float64, β::Float64, h₁::Float64, h₂::Float64, P::Vector{Float64}, W::Vector{Float64}) ::Matrix{Float64}
    # Inicializa a matriz local Kᵉ
    Kᵉ = zeros(4, 4)

    # Pré-calcula os fatores constantes das integrais
    cst1 = α * (h₂ / h₁)
    cst2 = α * (h₁ / h₂)
    cst3 = β * (h₁ * h₂ / 4)

    # Loop sobre as colunas (b) e linhas (a) da matriz local Kᵉ
    for b in 1:4
        for a in 1:4
            # Inicializa a contribuição da quadratura para a entrada (a, b)
            soma = 0.0

            # Integração via quadratura gaussiana dupla
            for i in 1:length(P)     # Loop sobre os pontos de quadratura no eixo ξ₁
				ξ₁ = P[i]
                for j in 1:length(P) # Loop sobre os pontos de quadratura no eixo ξ₂
                    ξ₂ = P[j]

                    # Acumula a contribuição da quadratura
                    soma += W[i] * W[j] * (
                        cst1 * ∂ϕ_∂ξ₁(ξ₁, ξ₂, b) * ∂ϕ_∂ξ₁(ξ₁, ξ₂, a) + 
                        cst2 * ∂ϕ_∂ξ₂(ξ₁, ξ₂, b) * ∂ϕ_∂ξ₂(ξ₁, ξ₂, a) + 
                        cst3 *      ϕ(ξ₁, ξ₂, b) *      ϕ(ξ₁, ξ₂, a)    
                    )
                end
            end

            # Armazena a contribuição acumulada na entrada [a, b] da matriz Kᵉ
            Kᵉ[a, b] = soma
        end
    end

    return Kᵉ
end

function kernel_monta_Kᵉ!(
    Kᵉ_flat::CuArray{Float64}, cst1::Float64, cst2::Float64, cst3::Float64,
    P, W, n_points::Int64
)
    idx = threadIdx().x + (blockIdx().x - 1) * blockDim().x

    # Each thread computes one (a, b) entry of Kᵉ
    if idx <= 16  # 4x4 elements
        a = div(idx - 1, 4) + 1
        b = rem(idx - 1, 4) + 1

        # Accumulate the quadrature contributions for Kᵉ[a, b]
        soma = 0.0
        for i in 1:n_points
            ξ₁ = P[i]
            w₁ = W[i]
            for j in 1:n_points
                ξ₂ = P[j]
                w₂ = W[j]

                # Compute the integrand contributions
                soma += w₁ * w₂ * (
                    cst1 * ∂ϕ_∂ξ₁(ξ₁, ξ₂, b) * ∂ϕ_∂ξ₁(ξ₁, ξ₂, a) + 
                    cst2 * ∂ϕ_∂ξ₂(ξ₁, ξ₂, b) * ∂ϕ_∂ξ₂(ξ₁, ξ₂, a) + 
                    cst3 *      ϕ(ξ₁, ξ₂, b) *      ϕ(ξ₁, ξ₂, a)
                )
            end
        end

        # Store the accumulated value in the flattened result
        Kᵉ_flat[idx] = soma
    end
end

function monta_Kᵉ_cuda_correct(α::Float64, β::Float64, h₁::Float64, h₂::Float64, P::CuVector{Float64,2}, W::CuVector{Float64,2}) :: CuArray{Float64, 2}
    # Precompute constants
    cst1 = α * (h₂ / h₁)
    cst2 = α * (h₁ / h₂)
    cst3 = β * (h₁ * h₂ / 4)

    # Number of quadrature points
    n_points = length(P)

    # Initialize the result matrix Kᵉ on the GPU
    Kᵉ = CUDA.zeros(Float64, 4, 4)

    # Flatten Kᵉ for efficient indexing in kernel
    Kᵉ_flat = CuArray(vec(Kᵉ))

    # Launch GPU kernel to compute Kᵉ
    @cuda threads=256 kernel_monta_Kᵉ!(
        Kᵉ_flat, cst1, cst2, cst3, 
        P, W, n_points
    )

    # Reshape the flattened result back to a 4x4 matrix
    return reshape(Kᵉ_flat, 4, 4)
    
end

monta_Kᵉ_cuda_correct (generic function with 1 method)

In [261]:
using Adapt 

function monta_Kᵉ_cuda(α::Float64, β::Float64, h₁::Float64, h₂::Float64, P, W)
    # Inicializa a matriz local Kᵉ
    Kᵉ = zeros(4, 4)

    # Pré-calcula os fatores constantes das integrais
    cst1 = α * (h₂ / h₁)
    cst2 = α * (h₁ / h₂)
    cst3 = β * (h₁ * h₂ / 4)

    # Loop sobre as colunas (b) e linhas (a) da matriz local Kᵉ
    for b in 1:4
        for a in 1:4
            # Inicializa a contribuição da quadratura para a entrada (a, b)
            soma = 0.0

            # Integração via quadratura gaussiana dupla
            for i in 1:length(P)     # Loop sobre os pontos de quadratura no eixo ξ₁
				ξ₁ = P[i]
                for j in 1:length(P) # Loop sobre os pontos de quadratura no eixo ξ₂
                    ξ₂ = P[j]

                    # Acumula a contribuição da quadratura
                    soma += W[i] * W[j] * (
                        cst1 * ∂ϕ_∂ξ₁(ξ₁, ξ₂, b) * ∂ϕ_∂ξ₁(ξ₁, ξ₂, a) + 
                        cst2 * ∂ϕ_∂ξ₂(ξ₁, ξ₂, b) * ∂ϕ_∂ξ₂(ξ₁, ξ₂, a) + 
                        cst3 *      ϕ(ξ₁, ξ₂, b) *      ϕ(ξ₁, ξ₂, a)    
                    )
                end
            end

            # Armazena a contribuição acumulada na entrada [a, b] da matriz Kᵉ
            Kᵉ[a, b] = soma
        end
    end

    Kᵉ_gpu = cu(Kᵉ)
    
    return Kᵉ_gpu
end

function teste_monta_Kᵉ_cuda()
	h₁ = 1/4
	h₂ = 1/4

	P, W = legendre(2)
    #P, W = CuArray(P), CuArray(W)

	# Teste 1
	α = 6.0
	β = 0.0
	Kᵉ = monta_Kᵉ_cuda(α, β, h₁, h₂, P, W)
	display("Kᵉ - Teste 1")
	display(Kᵉ)

	# Teste 2
	#α = 0.0
	#β = (9*4)/(h₁*h₂)
	#Kᵉ = monta_Kᵉ_cuda_correct(α, β, h₁, h₂, P, W)
	#display("Kᵉ - Teste 2")
	#display(Kᵉ)	
end
teste_monta_Kᵉ_cuda()

"Kᵉ - Teste 1"

4×4 CuArray{Float32, 2, CUDA.DeviceMemory}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [262]:
function teste_monta_Kᵉ()
	h₁ = 1/4
	h₂ = 1/4

	P, W = legendre(2)

	# Teste 1
	α = 6.0
	β = 0.0
	Kᵉ = monta_Kᵉ(α, β, h₁, h₂, P, W)
	display("Kᵉ - Teste 1")
	display(Kᵉ)

	# Teste 2
	α = 0.0
	β = (9*4)/(h₁*h₂)
	Kᵉ = monta_Kᵉ(α, β, h₁, h₂, P, W)
	display("Kᵉ - Teste 2")
	display(Kᵉ)	
end

teste_monta_Kᵉ (generic function with 1 method)

In [263]:
function teste_monta_Kᵉ_cuda()
	h₁ = 1/4
	h₂ = 1/4

	P, W = legendre(2)
    P, W = CuArray(P), CuArray(W)

	# Teste 1
	α = 6.0
	β = 0.0
	Kᵉ = monta_Kᵉ_cuda(α, β, h₁, h₂, P, W)
	display("Kᵉ - Teste 1")
	display(Kᵉ)

	# Teste 2
	#α = 0.0
	#β = (9*4)/(h₁*h₂)
	#Kᵉ = monta_Kᵉ_cuda_correct(α, β, h₁, h₂, P, W)
	#display("Kᵉ - Teste 2")
	#display(Kᵉ)	
end
teste_monta_Kᵉ_cuda()

"Kᵉ - Teste 1"

4×4 CuArray{Float32, 2, CUDA.DeviceMemory}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

# Montagem de LG e EQ

In [264]:
function monta_LG(Nx1::Int64, Nx2::Int64)::Matrix{Int64}
    # Define o número de funções φ no eixo x₁ e x₂
    nx1 = Nx1 + 1
    nx2 = Nx2 + 1

    # M[:,j] contém a numeração da primeira linha do "Bloco j"
    M = (1:nx1-1) .+ (0:nx1:(nx2-2)*nx1)'

    # LG[1,:] contém a numeração global da primeira função local de cada elemento 
    linha1 = reshape(M, 1, :)

    # Constrói a matriz LG
    LG = vcat(linha1, linha1 .+ 1, linha1 .+ (nx1+1), linha1 .+ nx1)

    return LG
end

monta_LG (generic function with 1 method)

In [265]:
function monta_EQ(Nx1::Int64, Nx2::Int64) :: Tuple{Int64,Vector{Int64}}
    # Define o número de funções φ no eixo x₁ e x₂
    nx1 = Nx1 + 1
    nx2 = Nx2 + 1
    
    # Calcula o número de funções globais φ que compõem a base do espaço Vₘ
    m = (nx1-2) * (nx2-2)

    # Inicializa o vetor EQ preenchido com m+1
    EQ = fill(m+1, nx1 * nx2)

    # Vetor contendo os índices das funções globais φ que compõem a base do espaço Vₘ
    L = reshape( (0:nx1-3) .+ (nx1+2:nx1:(nx2-2)*nx1+2)' , :,1)

    # Atribui os valores de 1 até m as funções globais φ que compõem a base do espaço Vₘ
    EQ[L] = 1:m

    return m, EQ
end

monta_EQ (generic function with 1 method)

# Montagem da F global

In [266]:
function monta_F(f::Function, Nx1::Int64, Nx2::Int64, m::Int64, EQoLG::Matrix{Int64})
    # Comprimento da base (h₁) e altura (h₂) de cada elemento retangular Ωᵉ
    h₁ = 1 / Nx1
    h₂ = 1 / Nx2

    # Número total de elementos finitos na malha
    ne = Nx1 * Nx2

    # P: Pontos de quadratura de Gauss-Legendre (ordem 5)
    # W: Pesos de quadratura de Gauss-Legendre
    P, W = legendre(5)

    # Inicializa o vetor local Fᵉ
    Fᵉ = zeros(4)

    # Inicializa o vetor global F com tamanho (m+1)
    F = zeros(m+1)
    
    # Loop sobre os elementos Ωᵉ (percorrendo cada subdivisão ao longo de x₂ e x₁)
    for j = 1:Nx2
        # Segunda componente do ponto inferior esquerdo do retângulo `Ωᵉ`.
        p₂ = (j-1)*h₂ 
        for i = 1:Nx1
            # Primeira componente do ponto inferior esquerdo do retângulo `Ωᵉ`.
            p₁ = (i-1)*h₁
            # Numeração do elemento finito atual (e) na malha
            e = (j-1)*Nx1 + i 

            # Calcula o vetor local Fᵉ para o elemento e
            monta_Fᵉ!(Fᵉ, f, h₁, h₂, p₁, p₂, P, W) 

            # Acumula Fᵉ em F usando a matriz de conectividade EQoLG
            for a = 1:4
                F[EQoLG[a,e]] += Fᵉ[a]
            end
        end
    end

    # Retorna o vetor global F com tamanho `m`, excluindo a última entrada adicional
    return F[1:m]
end


function monta_F_cuda(f::Function, Nx1::Int64, Nx2::Int64, m::Int64, EQoLG)
    # Comprimento da base (h₁) e altura (h₂) de cada elemento retangular Ωᵉ
    h₁ = 1 / Nx1
    h₂ = 1 / Nx2

    # Número total de elementos finitos na malha
    ne = Nx1 * Nx2

    # P: Pontos de quadratura de Gauss-Legendre (ordem 5)
    # W: Pesos de quadratura de Gauss-Legendre
    P, W = legendre(5)
    #P, W = CuArray(P), CuArray(W)

    # Inicializa o vetor local Fᵉ
    Fᵉ = CUDA.zeros(4)

    # Inicializa o vetor global F com tamanho (m+1)
    F = CUDA.zeros(m+1)
    
    # Loop sobre os elementos Ωᵉ (percorrendo cada subdivisão ao longo de x₂ e x₁)
    for j = 1:Nx2
        # Segunda componente do ponto inferior esquerdo do retângulo `Ωᵉ`.
        p₂ = (j-1)*h₂ 
        for i = 1:Nx1
            # Primeira componente do ponto inferior esquerdo do retângulo `Ωᵉ`.
            p₁ = (i-1)*h₁
            # Numeração do elemento finito atual (e) na malha
            e = (j-1)*Nx1 + i 

            # Calcula o vetor local Fᵉ para o elemento e
            monta_Fᵉ_cuda!(Fᵉ, f, h₁, h₂, p₁, p₂, P, W)

            # Acumula Fᵉ em F usando a matriz de conectividade EQoLG
            for a = 1:4
                F[EQoLG[a,e]] += Fᵉ[a]
            end
        end
    end

    # Retorna o vetor global F com tamanho `m`, excluindo a última entrada adicional
    return F[1:m]
end

monta_F_cuda (generic function with 1 method)

In [267]:
function teste_monta_F()
	# Teste 1
	#Nx1 = 4; Nx2 = 3; h₁ = 1/Nx1; h₂ = 1/Nx2;
	
	#m, EQ = monta_EQ(Nx1,Nx2)
	#LG = monta_LG(Nx1,Nx2)
	#EQoLG = EQ[LG]
	
	#F = monta_F((x₁,x₂) -> 4.0/(h₁*h₂), Nx1, Nx2, m, EQoLG)
    #display("F - Teste 1")
	#display("Nx1 = 4; Nx2 = 3; h₁ = 1/Nx1; h₂ = 1/Nx2; f(x₁,x₂) = 4/(h₁*h₂);")
	#display(F)
	
	# Teste 2
	Nx1 = 4; Nx2 = 4; h₁ = 1/Nx1; h₂ = 1/Nx2;
	
	m, EQ = monta_EQ(Nx1,Nx2)
	LG = monta_LG(Nx1,Nx2)
	EQoLG = EQ[LG]
	
	F = monta_F((x₁,x₂) -> (16*9*x₁*x₂)/((h₁*h₂)^2), Nx1, Nx2, m, EQoLG)
    display("F - Teste 2")
	display("Nx1 = Nx2 = 4; h₁ = 1/Nx1; h₂ = 1/Nx2; f(x₁,x₂) = (16*9*x₁*x₂)/((h₁*h₂)^2);")
	display(F)
end

teste_monta_F (generic function with 1 method)

In [268]:
teste_monta_F()

"F - Teste 2"

"Nx1 = Nx2 = 4; h₁ = 1/Nx1; h₂ = 1/Nx2; f(x₁,x₂) = (16*9*x₁*x₂)/((h₁*h₂)^2);"

9-element Vector{Float64}:
  143.99999999999994
  287.9999999999999
  431.9999999999998
  287.9999999999999
  575.9999999999999
  863.9999999999995
  431.9999999999998
  863.9999999999997
 1295.9999999999998

In [269]:
function teste_monta_F_cuda()
	# #Teste 1
	# Nx1 = 4; Nx2 = 3; h₁ = 1/Nx1; h₂ = 1/Nx2;
	
	# m, EQ = monta_EQ(Nx1,Nx2)
	# LG = monta_LG(Nx1,Nx2)
	# EQoLG = EQ[LG]
	
	# F = monta_F_cuda((x₁,x₂) -> 4.0/(h₁*h₂), Nx1, Nx2, m, EQoLG)
    # display("F - Teste 1")
	# display("Nx1 = 4; Nx2 = 3; h₁ = 1/Nx1; h₂ = 1/Nx2; f(x₁,x₂) = 4/(h₁*h₂);")
	# display(F)
	
	# Teste 2
	Nx1 = 4; Nx2 = 4; h₁ = 1/Nx1; h₂ = 1/Nx2;
	
	m, EQ = monta_EQ(Nx1,Nx2)
	LG = monta_LG(Nx1,Nx2)
	EQoLG = EQ[LG]
	
	F = monta_F_cuda((x₁,x₂) -> (16*9*x₁*x₂)/((h₁*h₂)^2), Nx1, Nx2, m, EQoLG)
    display("F - Teste 2")
	display("Nx1 = Nx2 = 4; h₁ = 1/Nx1; h₂ = 1/Nx2; f(x₁,x₂) = (16*9*x₁*x₂)/((h₁*h₂)^2);")
	display(F)
end

teste_monta_F_cuda (generic function with 1 method)

In [270]:
teste_monta_F_cuda()

"F - Teste 2"

"Nx1 = Nx2 = 4; h₁ = 1/Nx1; h₂ = 1/Nx2; f(x₁,x₂) = (16*9*x₁*x₂)/((h₁*h₂)^2);"

9-element CuArray{Float32, 1, CUDA.DeviceMemory}:
  144.0
  288.0
  432.0
  288.0
  576.0
  864.0
  432.0
  864.0
 1296.0

# Montagem da K global

In [271]:
function monta_K(α::Float64, β::Float64, Nx1::Int64, Nx2::Int64, m::Int64, EQoLG::Matrix{Int64}) :: SparseMatrixCSC{Float64, Int64}
    h₁ = 1 / Nx1 # Comprimento da base do elemento finito retangular Ωᵉ
    h₂ = 1 / Nx2 # Comprimento da altura do elemento finito retangular Ωᵉ

    # Número de elementos na malha
    ne = Nx1 * Nx2

    # Pontos e pesos de quadratura de Gauss-Legendre
    P, W = legendre(2)

    # Calcula a matriz local Kᵉ
    Kᵉ = monta_Kᵉ(α, β, h₁, h₂, P, W)

    # Inicializa a matriz esparsa K com tamanho (m+1) x (m+1)
    K = spzeros(m+1, m+1)

    # Loop sobre os elementos
    for e = 1:ne
        # Loop sobre as colunas (b) e linhas (a) da matriz local Kᵉ
        for b = 1:4
            j = EQoLG[b,e]
            for a = 1:4
                i = EQoLG[a,e]
                K[i,j] += Kᵉ[a,b]
            end
        end
    end
   
    # Remove a última linha e coluna da matriz K
    return K[1:m, 1:m]
end

monta_K (generic function with 1 method)

In [272]:
function teste_monta_K()
	α = 1.0; β = 1.0; Nx1 = 4; Nx2 = 3
	
	m, EQ = monta_EQ(Nx1,Nx2)
	LG = monta_LG(Nx1,Nx2)
	EQoLG = EQ[LG]
	
	K = monta_K(α, β, Nx1, Nx2, m, EQoLG)

	display("Parâmetros de entrada: α = 1.0; β = 1.0; Nx1 = 4; Nx2 = 3")
	display(K)
end

teste_monta_K (generic function with 1 method)

In [273]:
teste_monta_K()

"Parâmetros de entrada: α = 1.0; β = 1.0; Nx1 = 4; Nx2 = 3"

6×6 SparseMatrixCSC{Float64, Int64} with 28 stored entries:
  2.81481    -0.62963      ⋅         -0.0462963  -0.344907     ⋅ 
 -0.62963     2.81481    -0.62963    -0.344907   -0.0462963  -0.344907
   ⋅         -0.62963     2.81481      ⋅         -0.344907   -0.0462963
 -0.0462963  -0.344907     ⋅          2.81481    -0.62963      ⋅ 
 -0.344907   -0.0462963  -0.344907   -0.62963     2.81481    -0.62963
   ⋅         -0.344907   -0.0462963    ⋅         -0.62963     2.81481

In [274]:
function monta_K_cuda(α::Float64, β::Float64, Nx1::Int64, Nx2::Int64, m::Int64, EQoLG) :: SparseMatrixCSC{Float64, Int64}
    h₁ = 1 / Nx1 # Comprimento da base do elemento finito retangular Ωᵉ
    h₂ = 1 / Nx2 # Comprimento da altura do elemento finito retangular Ωᵉ

    # Número de elementos na malha
    ne = Nx1 * Nx2

    # Pontos e pesos de quadratura de Gauss-Legendre
    P, W = legendre(2)
    #P, W = CuArray(P), CuArray(W)

    # Calcula a matriz local Kᵉ
    Kᵉ = monta_Kᵉ_cuda(α, β, h₁, h₂, P, W)

    # Inicializa a matriz esparsa K com tamanho (m+1) x (m+1)
    K = spzeros(m+1, m+1)
    #K = CUDA.CUSPARSE.CuSparseMatrixCSC(K)

    # Loop sobre os elementos
    for e = 1:ne
        # Loop sobre as colunas (b) e linhas (a) da matriz local Kᵉ
        for b = 1:4
            j = EQoLG[b,e]
            for a = 1:4
                i = EQoLG[a,e]
                K[i,j] += Kᵉ[a,b]
            end
        end
    end
   
    # Remove a última linha e coluna da matriz K
    K = K[1:m, 1:m]
    K = cu(K) #CUDA.CUSPARSE.CuSparseMatrixCSC(K)
    return K# [1:m, 1:m]
end

function kernel(EQoLG, K, K_e, ne)
    # Each thread handles one element in the outer loop (over e)
    i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i <= ne
        for b = 1:4
            j = EQoLG[b, i]
            for a = 1:4
                k = EQoLG[a, i]
                K[k, j] += K_e[a, b]
            end
        end
    end
    return
end

# Launch the kernel
function parallelize_on_gpu(EQoLG_gpu, K_gpu, K_e_gpu, ne)
    # Define the number of threads per block and number of blocks
    threads_per_block = 8
    blocks = div(ne + threads_per_block - 1, threads_per_block)
    
    # Call the kernel
    @cuda threads=threads_per_block blocks=blocks kernel(EQoLG_gpu, K_gpu, K_e_gpu, ne)
end

# function teste_monta_K_cuda()
# 	α = 1.0; β = 1.0; Nx1 = 4; Nx2 = 3
	
# 	m, EQ = monta_EQ(Nx1,Nx2)
# 	LG = monta_LG(Nx1,Nx2)
# 	EQoLG = EQ[LG]
	
# 	K = monta_K_cuda(α, β, Nx1, Nx2, m, EQoLG)

# 	display("Parâmetros de entrada: α = 1.0; β = 1.0; Nx1 = 4; Nx2 = 3")
# 	display(K)
# end

function teste_monta_K_cuda()
	α = 1.0; β = 1.0; Nx1 = 4; Nx2 = 3
	
	m, EQ = monta_EQ(Nx1,Nx2)
	LG = monta_LG(Nx1,Nx2)
	EQoLG = EQ[LG]

	h₁ = 1 / Nx1 # Comprimento da base do elemento finito retangular Ωᵉ
    h₂ = 1 / Nx2 # Comprimento da altura do elemento finito retangular Ωᵉ

    # Número de elementos na malha
    ne = Nx1 * Nx2

    # Pontos e pesos de quadratura de Gauss-Legendre
    P, W = legendre(2)
    #P, W = CuArray(P), CuArray(W)

    # Calcula a matriz local Kᵉ
    Kᵉ = monta_Kᵉ_cuda(α, β, h₁, h₂, P, W)

    # Inicializa a matriz esparsa K com tamanho (m+1) x (m+1)
    K = spzeros(m+1, m+1)
    #K = CUDA.CUSPARSE.CuSparseMatrixCSC(K)

	parallelize_on_gpu(cu(EQoLG), cu(K), cu(Kᵉ), ne)
end

teste_monta_K_cuda()


CUDA.HostKernel for kernel(CuDeviceMatrix{Int64, 1}, CUDA.CUSPARSE.CuSparseDeviceMatrixCSC{Float32, Int32, 1}, CuDeviceMatrix{Float32, 1}, Int64)

In [275]:
teste_monta_K_cuda()

ERROR: a exception was thrown during kernel execution on thread (1, 1, 1) in block (1, 1, 1).
Stacktrace not available, run Julia on debug level 2 for more details (by passing -g2 to the executable).



CUDA.KernelException: KernelException: exception thrown during kernel execution on device NVIDIA GeForce RTX 3060 Ti

In [276]:
function erro_norma_L2(u::Function, c̄::Vector{Float64}, Nx1::Int64, Nx2::Int64, EQoLG::Matrix{Int64}) :: Float64
    # Comprimentos da base (h₁) e altura (h₂) de cada elemento retangular Ωᵉ
    h₁ = 1 / Nx1
    h₂ = 1 / Nx2

    # Inicializa o erro
    erro = 0.0

    # Define o número de pontos de quadratura (Npg) e os pontos (P) e pesos (W)
    Npg = 5
    P, W = legendre(Npg)

    # Loop sobre os elementos Ωᵉ (percorrendo cada subdivisão ao longo de x₂ e x₁)
    for j = 1:Nx2
        # Calcula a segunda componente do ponto inferior esquerdo do retângulo `Ωᵉ`.
        p₂ = (j - 1) * h₂ 
        for i = 1:Nx1
            # Calcula a primeira componente do ponto inferior esquerdo de `Ωᵉ`.
            p₁ = (i - 1) * h₁
            # Identifica o número do elemento finito atual (e) na malha
            e = (j - 1) * Nx1 + i

            # Obtém os coeficientes `c` da solução aproximada no elemento `e` 
            c1e = c̄[EQoLG[1, e]]
            c2e = c̄[EQoLG[2, e]]
            c3e = c̄[EQoLG[3, e]]
            c4e = c̄[EQoLG[4, e]]

            # Realiza a integração dupla usando quadratura de Gauss-Legendre
            for b = 1:Npg
                # Coordenada ξ₂ no espaço de referência e x₂ no espaço físico
                ξ₂ = P[b]
                x₂ = x₂_de_ξ(ξ₂, h₂, p₂)
                for a = 1:Npg
                    # Coordenada ξ₁ no espaço de referência e x₁ no espaço físico
                    ξ₁ = P[a]
                    x₁ = x₁_de_ξ(ξ₁, h₁, p₁)

                    # Acumula a contribuição da quadratura
                    erro += W[a] * W[b] * (u(x₁, x₂) - 
                        c1e * ϕ(ξ₁, ξ₂, 1) -
                        c2e * ϕ(ξ₁, ξ₂, 2) -
                        c3e * ϕ(ξ₁, ξ₂, 3) -
                        c4e * ϕ(ξ₁, ξ₂, 4))^2
                end
            end
        end
    end

    return sqrt(erro * h₁ * h₂ / 4)
end

erro_norma_L2 (generic function with 1 method)

In [277]:
function plot_solução_aproximada(c̄::Vector{Float64}, 
    Nx1::Int64, Nx2::Int64, EQoLG::Matrix{Int64})
# Comprimentos da base (h₁) e altura (h₂) de cada elemento retangular Ωᵉ
h₁, h₂ = 1 / Nx1, 1 / Nx2

# Define uma discretização do intervalo de referência [-1, 1] nos eixos ξ₁ e ξ₂
ξ₁ = collect(-1:0.1:1) 
ξ₂ = collect(-1:0.1:1) 

# Inicializa um objeto de gráfico que acumulará as superfícies
plt = Plots.plot(seriestype = :surface, title="Solução Aproximada",
color=:viridis,
xlabel="x₁", ylabel="x₂", colorbar=false, zlims=(0,1),
xticks=[0, 0.5, 1], yticks=[0, 0.5, 1], zticks=[0, 1])

# Loop nos elementos da malha (percorrendo cada subdivisão ao longo de x₂ e x₁)
for j = 1:Nx2
# Define a segunda coordenada do ponto inferior esquerdo do retângulo `Ωᵉ`.
p₂ = (j - 1) * h₂
# Calcula os valores correspondentes no eixo x₂ para os pontos ξ₂ em [-1,1]
x₂ = x₂_de_ξ.(ξ₂, h₂, p₂)

for i = 1:Nx1
# Primeira coordenada do ponto inferior esquerdo do retângulo `Ωᵉ`.
p₁ = (i - 1) * h₁
# Valores correspondentes no eixo x₁ para os pontos ξ₁ em [-1,1]
x₁ = x₁_de_ξ.(ξ₁, h₁, p₁)

# Determina a numeração do elemento finito atual (`e`) na malha
e = (j - 1) * Nx1 + i

# Obtém os coeficientes `c` da solução aproximada no elemento `e`
ce = c̄[EQoLG[:, e]]

# Solução aproximada na malha x₁ × x₂. Tamanho length(x₂) x length(x₁)
mat = [dot(ce,ϕ(ξ₁[a],ξ₂[b])) for b in 1:length(x₂), a in 1:length(x₁)]

# Exibindo os resultados
# display("------------------------------")
# display("e = $e")
# display("p1, p2 = $p₁, $p₂")
# display("ce = $ce")
# display("x₁ = $x₁")
# display("x₂ = $x₂")
# display("uₕ(x₁[a],x₂[b]) for b in 1:length(x₂), a in 1:length(x₁)")
# display(mat)

# Gera o gráfico da solução aproximada sobre o elemento `e`
Plots.plot!(plt, x₁, x₂, mat, seriestype = :surface, alpha=0.5)
end
end

return plt
end

plot_solução_aproximada (generic function with 1 method)

In [278]:
function estudo_do_erro()
	# Carrega os dados de entrada da EDP
    α, β, f, u = exemplo1()

    # Define os valores para Nx1 (subdivisões em x₁) e Nx2 (subdivisões em x₂)
    vec_Nx1 = [2^i for i in 2:9]
    vec_Nx2 = vec_Nx1

    # Calcula os valores de `h` para cada combinação de vec_Nx1[i] e vec_Nx2[i]
    vec_h = sqrt.( (1 ./ vec_Nx1).^2 + (1 ./ vec_Nx2).^2 )

    # Inicializa o vetor para armazenar os erros da norma L2
    vec_erro = zeros(length(vec_Nx1))

    # Loop sobre os valores de Nx1 e Nx2
    for i = 1:length(vec_Nx1)
        Nx1 = vec_Nx1[i]
        Nx2 = vec_Nx2[i]

        # Gera a matriz de conectividade local/global (LG)
        LG = monta_LG(Nx1, Nx2)

        # Gera o valor de `m` e o vetor `EQ`
        m, EQ = monta_EQ(Nx1, Nx2)

        # Define a matriz de conectividade EQoLG
        EQoLG = EQ[LG]

        # Monta a matriz esparsa `K`
        @time begin
        K = monta_K(α, β, Nx1, Nx2, m, EQoLG)
        # Monta o vetor global `F`
        F = monta_F(f, Nx1, Nx2, m, EQoLG)

        # Resolve o sistema linear Kc = F
        c = K \ F
        end

        # Calcula o erro na norma L2 entre a solução exata `u` e a solução aproximada
        vec_erro[i] = erro_norma_L2(u, [c;0], Nx1, Nx2, EQoLG)
    end

    return vec_h, vec_erro
end

estudo_do_erro (generic function with 1 method)

In [279]:
function plot_estudo_do_erro()
	# Realiza o estudo do erro e mede o tempo de execução
	@time begin
		vec_h, vec_erro = estudo_do_erro()
	end

    # Cria o gráfico do erro na norma L2 em função de h, diâmetro de cada Ωᵉ
    plt = Plots.plot(
        vec_h, vec_erro, lw=3, linestyle=:solid, markershape=:circle,
        label="Erro", title="Estudo do erro",
        xscale=:log10, yscale=:log10, legend=:topleft
    )

    # Adiciona a curva teórica de h² ao gráfico
    Plots.plot!(plt, vec_h, vec_h.^2, lw=3, linestyle=:solid, label="h²")

    # Adiciona rótulos aos eixos
    Plots.xlabel!("h")
    Plots.ylabel!("Erro")

    # Exibe uma tabela com os valores de h e erro
    display("Tabela com os valores de h e erro:")
    display(DataFrame(h=vec_h, erro=vec_erro))

	# Retorna o gráfico 
	return plt
end


plot_estudo_do_erro (generic function with 1 method)

In [280]:
function estudo_do_erro_cuda()
	# Carrega os dados de entrada da EDP
    α, β, f, u = exemplo1()

    # Define os valores para Nx1 (subdivisões em x₁) e Nx2 (subdivisões em x₂)
    vec_Nx1 = [2^i for i in 2:9]
    vec_Nx2 = vec_Nx1

    # Calcula os valores de `h` para cada combinação de vec_Nx1[i] e vec_Nx2[i]
    vec_h = sqrt.( (1 ./ vec_Nx1).^2 + (1 ./ vec_Nx2).^2 )

    # Inicializa o vetor para armazenar os erros da norma L2
    vec_erro = zeros(length(vec_Nx1))

    # Loop sobre os valores de Nx1 e Nx2
    for i = 1:length(vec_Nx1)
        Nx1 = vec_Nx1[i]
        Nx2 = vec_Nx2[i]

        # Gera a matriz de conectividade local/global (LG)
        LG = monta_LG(Nx1, Nx2)

        # Gera o valor de `m` e o vetor `EQ`
        m, EQ = monta_EQ(Nx1, Nx2)

        # Define a matriz de conectividade EQoLG
        EQoLG = EQ[LG]

        # Monta a matriz esparsa `K`
        @time begin
        K = monta_K(α, β, Nx1, Nx2, m, EQoLG)
        # Monta o vetor global `F`
        F = monta_F(f, Nx1, Nx2, m, EQoLG)

        # Resolve o sistema linear Kc = F
        c = cu(K) \ cu(F)
        end

        # Calcula o erro na norma L2 entre a solução exata `u` e a solução aproximada
        vec_erro[i] = erro_norma_L2(u, [c;0], Nx1, Nx2, EQoLG)
    end

    return vec_h, vec_erro
end

estudo_do_erro_cuda (generic function with 1 method)

In [281]:
function plot_estudo_do_erro_cuda()
	# Realiza o estudo do erro e mede o tempo de execução
	@time begin
		vec_h, vec_erro = estudo_do_erro()
	end

    # Cria o gráfico do erro na norma L2 em função de h, diâmetro de cada Ωᵉ
    plt = Plots.plot(
        vec_h, vec_erro, lw=3, linestyle=:solid, markershape=:circle,
        label="Erro", title="Estudo do erro",
        xscale=:log10, yscale=:log10, legend=:topleft
    )

    # Adiciona a curva teórica de h² ao gráfico
    Plots.plot!(plt, vec_h, vec_h.^2, lw=3, linestyle=:solid, label="h²")

    # Adiciona rótulos aos eixos
    Plots.xlabel!("h")
    Plots.ylabel!("Erro")

    # Exibe uma tabela com os valores de h e erro
    display("Tabela com os valores de h e erro:")
    display(DataFrame(h=vec_h, erro=vec_erro))

	# Retorna o gráfico 
	return plt
end

plot_estudo_do_erro_cuda (generic function with 1 method)

In [282]:
plot_estudo_do_erro()

  0.023309 seconds (2.40 k allocations: 124.016 KiB, 98.71% compilation time)
  0.000412 seconds (513 allocations: 126.891 KiB)
  0.000860 seconds (1.68 k allocations: 600.766 KiB)
  0.003036 seconds (6.31 k allocations: 2.732 MiB)
  0.024097 seconds (24.77 k allocations: 10.418 MiB, 35.58% gc time)
  0.220467 seconds (98.50 k allocations: 45.287 MiB, 4.45% gc time)
  2.555156 seconds (393.42 k allocations: 194.068 MiB, 0.71% gc time)


InterruptException: InterruptException:

In [283]:
plot_estudo_do_erro_cuda()

  0.000231 seconds (217 allocations: 30.406 KiB)
  0.000251 seconds (513 allocations: 126.891 KiB)
  0.000654 seconds (1.68 k allocations: 600.766 KiB)
  0.002596 seconds (6.31 k allocations: 2.732 MiB)
  0.017272 seconds (24.75 k allocations: 10.418 MiB)
  0.255512 seconds (98.50 k allocations: 45.287 MiB, 18.94% gc time)
  2.790905 seconds (393.42 k allocations: 194.068 MiB, 10.04% gc time)


InterruptException: InterruptException:

In [284]:
a = CUDA.zeros(2, 3)

2×3 CuArray{Float32, 2, CUDA.DeviceMemory}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [285]:
using CUDA 

const n = 10 # 1048576, number of elements in 1D arrays
const THREADS_PER_BLOCK = 5

# create a vector [0.0, 0.0, 0.0...], and send to GPU
C = zeros(Float32, n) |> cu 

# create two vectors, fill them with 1s and 2s, and send to GPU
A = fill(1.0f0, n) |> cu 
B = fill(2.0f0, n) |> cu

function add!(c, a, b)
    # compute the thread_id
    if threadIdx().x == 1
        for i = 1:5
            @inbounds c[i] = 100
        end
    end
    
    x = (blockIdx().x - 1) * blockDim().x + threadIdx().x 

    # i'th thread should get the i'th elements of a and b 
    # sum them, and then store them in i'th element of c
    @inbounds c[x] = a[x] + b[x]
    return
end

threads = THREADS_PER_BLOCK # 256
blocks = n ÷ threads # 4096

# launch the kernel with 4096 blocks, of 256 threads each
@cuda threads=threads blocks=blocks add!(C, A, B)

CUDA.HostKernel for add!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [286]:
C

10-element CuArray{Float32, 1, CUDA.DeviceMemory}:
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0
 3.0